In [147]:
from dotenv import load_dotenv
from labelbox import Client, Project, Dataset
from typing import Dict
import uuid

### Setting Labelbox Connection

In [148]:
load_dotenv('labelbox3.env')
client = Client()

### Get Project Configuration

In [149]:
projects = client.get_projects(where=(Project.name == "RebootRx_pre-annot"))
project = next(iter(projects))
print(project.name, project.uid)

RebootRx_pre-annot ckl1yctgwyl2n07324ue230xf


### To Get Available Datasets

In [150]:
for _datasets in project.datasets():
    print(_datasets.uid, _datasets.name)

ckl22jq9603ya0y8w0zpz7fog first-150-2


### To Get Annotated Datasets

In [151]:
current_labels = []
for _labels in project.labels():
    print(_labels.uid)
print(current_labels)

[]


### Check All Available Data

In [195]:
data = next(iter(project.datasets()))
rows = list(data.data_rows())
len(rows)

150

In [208]:
def get_current_import_requests(project_id):
    response = client.execute(
                    """
                    query get_all_import_requests(
                        $project_id : ID! 
                    ) {
                      bulkImportRequests(where: {projectId: $project_id}) {
                        id
                        name
                        state
                      }
                    }
                    """,
                    {"project_id": project_id})
    
    return response['bulkImportRequests']

In [209]:
def delete_import_request(import_request_id):
    response = client.execute(
                    """
                        mutation delete_import_request(
                            $import_request_id : ID! 
                        ){
                          deleteBulkImportRequest(where: {id: $import_request_id}) {
                            id
                            name
                          }
                        }
                    """,
                    {"import_request_id": import_request_id})
    
    return response

In [210]:
def get_ontology(client: Client, project_id: str) -> Dict[str, str]:
  result = client.execute("""
    query get_ontology($proj_id: ID!) {
      project(where: {id: $proj_id}) {
        ontology {
          normalized
          }
          }
      }
    """, 
    {"proj_id": project_id})
  return result['project']['ontology']['normalized']

In [212]:
def get_entity_schema_id(ontology, name):
  tools = ontology['tools']
  return next((t for t in tools if t["name"] == name), None)

In [216]:
ontology= get_ontology(client, project.uid)
ontology

{'tools': [{'schemaNodeId': 'ckjrshew80b7h0y7gg1g52ad4',
   'featureSchemaId': 'ckjrshew80b7i0y7g9zzydsj4',
   'required': False,
   'name': 'group1',
   'tool': 'named-entity',
   'color': '#FF34FF',
   'classifications': []},
  {'schemaNodeId': 'ckjrshew80b7j0y7gdq6za1zc',
   'featureSchemaId': 'ckjrshew80b7k0y7gaubc7la9',
   'required': False,
   'name': 'group2',
   'tool': 'named-entity',
   'color': '#FF4A46',
   'classifications': []},
  {'schemaNodeId': 'ckjrsivld09yc0y9ghr70fisq',
   'featureSchemaId': 'ckjrsj79q0d550y83cx113wp1',
   'required': False,
   'name': 'total_sample_size',
   'tool': 'named-entity',
   'color': '#006FA6',
   'classifications': []},
  {'schemaNodeId': 'ckjrsivld09ye0y9gd8k138hw',
   'featureSchemaId': 'ckjrsj79r0d560y83fv2875ab',
   'required': False,
   'name': 'g1_n',
   'tool': 'named-entity',
   'color': '#A30059',
   'classifications': []},
  {'schemaNodeId': 'ckjrsivle09yg0y9g3vuydlmb',
   'featureSchemaId': 'ckjrsj79r0d570y83aw4xg9t7',
   'req

In [218]:
get_entity_schema_id(ontology, 'group1')

{'schemaNodeId': 'ckjrshew80b7h0y7gg1g52ad4',
 'featureSchemaId': 'ckjrshew80b7i0y7g9zzydsj4',
 'required': False,
 'name': 'group1',
 'tool': 'named-entity',
 'color': '#FF34FF',
 'classifications': []}

### Getting PMID Row ID

In [213]:
PMIDS = {}

In [214]:
for _rows in rows:
    row_data = _rows.row_data.split(" ")
    current_pmid = row_data[0].replace("PMID:", "")
    PMIDS[current_pmid] = _rows.uid
len(PMIDS)

150

### Format Upload Annotation

In [215]:
sample_request = {
    "data": [
        {
            "tag": "total_sample_size",
            "row_uid": "33278301",
            "location": {
                "start": 762,
                "end": 765
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "33278301",
            "location": {
                "start": 1183,
                "end": 1185
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "33278301",
            "location": {
                "start": 1208,
                "end": 1210
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "33225411",
            "location": {
                "start": 1107,
                "end": 1119
            }
        },
        {
            "tag": "group1",
            "row_uid": "32642699",
            "location": {
                "start": 577,
                "end": 595
            }
        },
        {
            "tag": "group2",
            "row_uid": "32642699",
            "location": {
                "start": 577,
                "end": 595
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32642699",
            "location": {
                "start": 1162,
                "end": 1181
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "32642699",
            "location": {
                "start": 1310,
                "end": 1315
            }
        },
        {
            "tag": "g2_response_rate",
            "row_uid": "32642699",
            "location": {
                "start": 1386,
                "end": 1391
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "32642699",
            "location": {
                "start": 1457,
                "end": 1460
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "32642699",
            "location": {
                "start": 1526,
                "end": 1531
            }
        },
        {
            "tag": "g2_median_OS",
            "row_uid": "32642699",
            "location": {
                "start": 1465,
                "end": 1476
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "32642699",
            "location": {
                "start": 1555,
                "end": 1560
            }
        },
        {
            "tag": "g1_survival_rate",
            "row_uid": "32642699",
            "location": {
                "start": 1310,
                "end": 1315
            }
        },
        {
            "tag": "group1",
            "row_uid": "32156749",
            "location": {
                "start": 636,
                "end": 671
            }
        },
        {
            "tag": "group2",
            "row_uid": "32156749",
            "location": {
                "start": 686,
                "end": 711
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32156749",
            "location": {
                "start": 1018,
                "end": 1038
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "32156749",
            "location": {
                "start": 1018,
                "end": 1029
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "32156749",
            "location": {
                "start": 1058,
                "end": 1060
            }
        },
        {
            "tag": "response_p",
            "row_uid": "32156749",
            "location": {
                "start": 1207,
                "end": 1214
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "32156749",
            "location": {
                "start": 1686,
                "end": 1690
            }
        },
        {
            "tag": "group1",
            "row_uid": "31925317",
            "location": {
                "start": 576,
                "end": 618
            }
        },
        {
            "tag": "group2",
            "row_uid": "31925317",
            "location": {
                "start": 633,
                "end": 641
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31925317",
            "location": {
                "start": 734,
                "end": 752
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "31925317",
            "location": {
                "start": 1204,
                "end": 1207
            }
        },
        {
            "tag": "g2_response_rate",
            "row_uid": "31925317",
            "location": {
                "start": 1216,
                "end": 1219
            }
        },
        {
            "tag": "response_p",
            "row_uid": "31925317",
            "location": {
                "start": 1177,
                "end": 1181
            }
        },
        {
            "tag": "group1",
            "row_uid": "31521049",
            "location": {
                "start": 688,
                "end": 778
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31521049",
            "location": {
                "start": 901,
                "end": 903
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31521049",
            "location": {
                "start": 914,
                "end": 915
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "31521049",
            "location": {
                "start": 932,
                "end": 934
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "31521049",
            "location": {
                "start": 1259,
                "end": 1262
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "31521049",
            "location": {
                "start": 1430,
                "end": 1440
            }
        },
        {
            "tag": "group1",
            "row_uid": "31463691",
            "location": {
                "start": 401,
                "end": 408
            }
        },
        {
            "tag": "group2",
            "row_uid": "31463691",
            "location": {
                "start": 293,
                "end": 317
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31463691",
            "location": {
                "start": 478,
                "end": 498
            }
        },
        {
            "tag": "group1",
            "row_uid": "31120501",
            "location": {
                "start": 1595,
                "end": 1606
            }
        },
        {
            "tag": "group2",
            "row_uid": "31120501",
            "location": {
                "start": 1248,
                "end": 1259
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31120501",
            "location": {
                "start": 1473,
                "end": 1476
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31120501",
            "location": {
                "start": 1565,
                "end": 1567
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "31120501",
            "location": {
                "start": 1612,
                "end": 1614
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "31120501",
            "location": {
                "start": 2076,
                "end": 2081
            }
        },
        {
            "tag": "g2_response_rate",
            "row_uid": "31120501",
            "location": {
                "start": 2112,
                "end": 2117
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "31120501",
            "location": {
                "start": 1931,
                "end": 1942
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "31120501",
            "location": {
                "start": 1788,
                "end": 1798
            }
        },
        {
            "tag": "g2_median_OS",
            "row_uid": "31120501",
            "location": {
                "start": 1990,
                "end": 2001
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "31120501",
            "location": {
                "start": 1845,
                "end": 1855
            }
        },
        {
            "tag": "g1_survival_rate",
            "row_uid": "31120501",
            "location": {
                "start": 1664,
                "end": 1667
            }
        },
        {
            "tag": "g2_survival_rate",
            "row_uid": "31120501",
            "location": {
                "start": 1707,
                "end": 1710
            }
        },
        {
            "tag": "response_p",
            "row_uid": "31120501",
            "location": {
                "start": 2153,
                "end": 2157
            }
        },
        {
            "tag": "group1",
            "row_uid": "30635337",
            "location": {
                "start": 712,
                "end": 722
            }
        },
        {
            "tag": "group2",
            "row_uid": "30635337",
            "location": {
                "start": 739,
                "end": 757
            }
        },
        {
            "tag": "group1",
            "row_uid": "30603055",
            "location": {
                "start": 638,
                "end": 641
            }
        },
        {
            "tag": "group1",
            "row_uid": "30431566",
            "location": {
                "start": 1119,
                "end": 1153
            }
        },
        {
            "tag": "group2",
            "row_uid": "30431566",
            "location": {
                "start": 1318,
                "end": 1348
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30431566",
            "location": {
                "start": 1075,
                "end": 1078
            }
        },
        {
            "tag": "response_p",
            "row_uid": "30431566",
            "location": {
                "start": 1458,
                "end": 1462
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "30431566",
            "location": {
                "start": 1547,
                "end": 1551
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "30431566",
            "location": {
                "start": 1503,
                "end": 1507
            }
        },
        {
            "tag": "group1",
            "row_uid": "29848757",
            "location": {
                "start": 507,
                "end": 539
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "29848757",
            "location": {
                "start": 446,
                "end": 448
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "27463016",
            "location": {
                "start": 1235,
                "end": 1250
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "27463016",
            "location": {
                "start": 1330,
                "end": 1340
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "27463016",
            "location": {
                "start": 1308,
                "end": 1318
            }
        },
        {
            "tag": "g2_median_OS",
            "row_uid": "27463016",
            "location": {
                "start": 1330,
                "end": 1340
            }
        },
        {
            "tag": "group1",
            "row_uid": "26375670",
            "location": {
                "start": 702,
                "end": 714
            }
        },
        {
            "tag": "group2",
            "row_uid": "26375670",
            "location": {
                "start": 719,
                "end": 732
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "26375670",
            "location": {
                "start": 597,
                "end": 600
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "25944689",
            "location": {
                "start": 723,
                "end": 734
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "25944689",
            "location": {
                "start": 1393,
                "end": 1397
            }
        },
        {
            "tag": "group1",
            "row_uid": "25905586",
            "location": {
                "start": 529,
                "end": 553
            }
        },
        {
            "tag": "group2",
            "row_uid": "25905586",
            "location": {
                "start": 558,
                "end": 569
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "25905586",
            "location": {
                "start": 843,
                "end": 863
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "25905586",
            "location": {
                "start": 1162,
                "end": 1173
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "25905586",
            "location": {
                "start": 1472,
                "end": 1476
            }
        },
        {
            "tag": "group1",
            "row_uid": "24991840",
            "location": {
                "start": 724,
                "end": 727
            }
        },
        {
            "tag": "group2",
            "row_uid": "24991840",
            "location": {
                "start": 521,
                "end": 539
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "24991840",
            "location": {
                "start": 1033,
                "end": 1040
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "24991840",
            "location": {
                "start": 1320,
                "end": 1328
            }
        },
        {
            "tag": "g1_survival_rate",
            "row_uid": "24991840",
            "location": {
                "start": 1373,
                "end": 1377
            }
        },
        {
            "tag": "g2_survival_rate",
            "row_uid": "24991840",
            "location": {
                "start": 1387,
                "end": 1391
            }
        },
        {
            "tag": "group1",
            "row_uid": "24991839",
            "location": {
                "start": 447,
                "end": 502
            }
        },
        {
            "tag": "group2",
            "row_uid": "24991839",
            "location": {
                "start": 471,
                "end": 502
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "24991839",
            "location": {
                "start": 549,
                "end": 563
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "24991839",
            "location": {
                "start": 549,
                "end": 563
            }
        },
        {
            "tag": "group1",
            "row_uid": "24991838",
            "location": {
                "start": 225,
                "end": 293
            }
        },
        {
            "tag": "group2",
            "row_uid": "24991838",
            "location": {
                "start": 268,
                "end": 293
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "24991838",
            "location": {
                "start": 592,
                "end": 594
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "24991838",
            "location": {
                "start": 1383,
                "end": 1389
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "24991836",
            "location": {
                "start": 1330,
                "end": 1332
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "24991836",
            "location": {
                "start": 1527,
                "end": 1533
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "24991836",
            "location": {
                "start": 1581,
                "end": 1585
            }
        },
        {
            "tag": "group1",
            "row_uid": "24991835",
            "location": {
                "start": 913,
                "end": 927
            }
        },
        {
            "tag": "group2",
            "row_uid": "24991835",
            "location": {
                "start": 976,
                "end": 989
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "24991835",
            "location": {
                "start": 788,
                "end": 790
            }
        },
        {
            "tag": "group1",
            "row_uid": "24991834",
            "location": {
                "start": 601,
                "end": 634
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "24991834",
            "location": {
                "start": 683,
                "end": 685
            }
        },
        {
            "tag": "group1",
            "row_uid": "24821822",
            "location": {
                "start": 387,
                "end": 411
            }
        },
        {
            "tag": "group2",
            "row_uid": "24821822",
            "location": {
                "start": 656,
                "end": 662
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "24821822",
            "location": {
                "start": 944,
                "end": 946
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "24821822",
            "location": {
                "start": 649,
                "end": 651
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "24821822",
            "location": {
                "start": 668,
                "end": 670
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "24821822",
            "location": {
                "start": 1066,
                "end": 1070
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "24821822",
            "location": {
                "start": 1057,
                "end": 1061
            }
        },
        {
            "tag": "g2_median_OS",
            "row_uid": "24821822",
            "location": {
                "start": 1066,
                "end": 1070
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "24821822",
            "location": {
                "start": 1057,
                "end": 1061
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "23873432",
            "location": {
                "start": 281,
                "end": 304
            }
        },
        {
            "tag": "g1_survival_rate",
            "row_uid": "23873432",
            "location": {
                "start": 643,
                "end": 649
            }
        },
        {
            "tag": "g2_survival_rate",
            "row_uid": "23873432",
            "location": {
                "start": 667,
                "end": 673
            }
        },
        {
            "tag": "group1",
            "row_uid": "22878749",
            "location": {
                "start": 650,
                "end": 653
            }
        },
        {
            "tag": "group2",
            "row_uid": "22878749",
            "location": {
                "start": 674,
                "end": 692
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "22878749",
            "location": {
                "start": 603,
                "end": 624
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "22878749",
            "location": {
                "start": 639,
                "end": 644
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "22878749",
            "location": {
                "start": 666,
                "end": 668
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "22878749",
            "location": {
                "start": 1371,
                "end": 1373
            }
        },
        {
            "tag": "group1",
            "row_uid": "31800611",
            "location": {
                "start": 659,
                "end": 681
            }
        },
        {
            "tag": "group2",
            "row_uid": "31800611",
            "location": {
                "start": 687,
                "end": 694
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31800611",
            "location": {
                "start": 1078,
                "end": 1081
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31800611",
            "location": {
                "start": 1123,
                "end": 1125
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "31800611",
            "location": {
                "start": 1143,
                "end": 1146
            }
        },
        {
            "tag": "g2_survival_rate",
            "row_uid": "31800611",
            "location": {
                "start": 1415,
                "end": 1420
            }
        },
        {
            "tag": "group1",
            "row_uid": "29718396",
            "location": {
                "start": 813,
                "end": 822
            }
        },
        {
            "tag": "group2",
            "row_uid": "29718396",
            "location": {
                "start": 858,
                "end": 868
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "29718396",
            "location": {
                "start": 758,
                "end": 761
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "29718396",
            "location": {
                "start": 828,
                "end": 831
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "29718396",
            "location": {
                "start": 888,
                "end": 891
            }
        },
        {
            "tag": "group1",
            "row_uid": "25756351",
            "location": {
                "start": 568,
                "end": 625
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "25756351",
            "location": {
                "start": 794,
                "end": 807
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "25756351",
            "location": {
                "start": 1314,
                "end": 1317
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "25756351",
            "location": {
                "start": 1630,
                "end": 1634
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "25756351",
            "location": {
                "start": 1559,
                "end": 1563
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "25756351",
            "location": {
                "start": 1660,
                "end": 1666
            }
        },
        {
            "tag": "group1",
            "row_uid": "24464611",
            "location": {
                "start": 945,
                "end": 977
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "24464611",
            "location": {
                "start": 799,
                "end": 802
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "24464611",
            "location": {
                "start": 799,
                "end": 802
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "24464611",
            "location": {
                "start": 1131,
                "end": 1135
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "24464611",
            "location": {
                "start": 1293,
                "end": 1297
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "24464611",
            "location": {
                "start": 1366,
                "end": 1371
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "24464611",
            "location": {
                "start": 1321,
                "end": 1325
            }
        },
        {
            "tag": "Other_HR",
            "row_uid": "24464611",
            "location": {
                "start": 1293,
                "end": 1297
            }
        },
        {
            "tag": "group1",
            "row_uid": "23977366",
            "location": {
                "start": 595,
                "end": 715
            }
        },
        {
            "tag": "group2",
            "row_uid": "23977366",
            "location": {
                "start": 731,
                "end": 773
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "23977366",
            "location": {
                "start": 512,
                "end": 515
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "23977366",
            "location": {
                "start": 719,
                "end": 721
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "23977366",
            "location": {
                "start": 777,
                "end": 779
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "23977366",
            "location": {
                "start": 1377,
                "end": 1381
            }
        },
        {
            "tag": "group1",
            "row_uid": "22622810",
            "location": {
                "start": 1102,
                "end": 1112
            }
        },
        {
            "tag": "group2",
            "row_uid": "22622810",
            "location": {
                "start": 1117,
                "end": 1146
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "22622810",
            "location": {
                "start": 951,
                "end": 954
            }
        },
        {
            "tag": "group1",
            "row_uid": "20683253",
            "location": {
                "start": 456,
                "end": 798
            }
        },
        {
            "tag": "group2",
            "row_uid": "20683253",
            "location": {
                "start": 880,
                "end": 941
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "20683253",
            "location": {
                "start": 826,
                "end": 829
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "20683253",
            "location": {
                "start": 970,
                "end": 973
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "20683253",
            "location": {
                "start": 1510,
                "end": 1514
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "20683253",
            "location": {
                "start": 1554,
                "end": 1559
            }
        },
        {
            "tag": "group1",
            "row_uid": "33313149",
            "location": {
                "start": 335,
                "end": 368
            }
        },
        {
            "tag": "group2",
            "row_uid": "33313149",
            "location": {
                "start": 1329,
                "end": 1336
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "33313149",
            "location": {
                "start": 1013,
                "end": 1026
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "33313149",
            "location": {
                "start": 1251,
                "end": 1254
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "33313149",
            "location": {
                "start": 1317,
                "end": 1320
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "33313149",
            "location": {
                "start": 1241,
                "end": 1246
            }
        },
        {
            "tag": "g2_response_rate",
            "row_uid": "33313149",
            "location": {
                "start": 1241,
                "end": 1312
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "33313149",
            "location": {
                "start": 1557,
                "end": 1561
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "33313149",
            "location": {
                "start": 1584,
                "end": 1589
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "33313149",
            "location": {
                "start": 1626,
                "end": 1631
            }
        },
        {
            "tag": "group1",
            "row_uid": "33278671",
            "location": {
                "start": 667,
                "end": 676
            }
        },
        {
            "tag": "group2",
            "row_uid": "33278671",
            "location": {
                "start": 702,
                "end": 707
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "33278671",
            "location": {
                "start": 504,
                "end": 507
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "33278671",
            "location": {
                "start": 1556,
                "end": 1560
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "33262518",
            "location": {
                "start": 288,
                "end": 294
            }
        },
        {
            "tag": "group1",
            "row_uid": "33243068",
            "location": {
                "start": 902,
                "end": 932
            }
        },
        {
            "tag": "group2",
            "row_uid": "33243068",
            "location": {
                "start": 966,
                "end": 1002
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "33243068",
            "location": {
                "start": 608,
                "end": 611
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "33243068",
            "location": {
                "start": 657,
                "end": 659
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "33243068",
            "location": {
                "start": 779,
                "end": 781
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "33243068",
            "location": {
                "start": 1497,
                "end": 1502
            }
        },
        {
            "tag": "group1",
            "row_uid": "33223758",
            "location": {
                "start": 742,
                "end": 750
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "33223758",
            "location": {
                "start": 711,
                "end": 716
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "33223758",
            "location": {
                "start": 808,
                "end": 811
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "33223758",
            "location": {
                "start": 755,
                "end": 757
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "33223758",
            "location": {
                "start": 1056,
                "end": 1061
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "33223758",
            "location": {
                "start": 1958,
                "end": 1963
            }
        },
        {
            "tag": "g1_survival_rate",
            "row_uid": "33223758",
            "location": {
                "start": 986,
                "end": 993
            }
        },
        {
            "tag": "g2_survival_rate",
            "row_uid": "33223758",
            "location": {
                "start": 1245,
                "end": 1251
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "33223758",
            "location": {
                "start": 1988,
                "end": 1993
            }
        },
        {
            "tag": "Other_HR",
            "row_uid": "33223758",
            "location": {
                "start": 1625,
                "end": 1630
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "33212778",
            "location": {
                "start": 781,
                "end": 783
            }
        },
        {
            "tag": "group1",
            "row_uid": "33212505",
            "location": {
                "start": 392,
                "end": 461
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "33212505",
            "location": {
                "start": 586,
                "end": 588
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "33210440",
            "location": {
                "start": 982,
                "end": 994
            }
        },
        {
            "tag": "group1",
            "row_uid": "33194743",
            "location": {
                "start": 175,
                "end": 184
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "33194743",
            "location": {
                "start": 572,
                "end": 579
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "33029097",
            "location": {
                "start": 720,
                "end": 732
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "33029097",
            "location": {
                "start": 1292,
                "end": 1296
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "33029097",
            "location": {
                "start": 1244,
                "end": 1248
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "33029097",
            "location": {
                "start": 1321,
                "end": 1325
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "33029097",
            "location": {
                "start": 1273,
                "end": 1277
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32980534",
            "location": {
                "start": 512,
                "end": 525
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "32980534",
            "location": {
                "start": 649,
                "end": 652
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32972040",
            "location": {
                "start": 391,
                "end": 396
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32925714",
            "location": {
                "start": 512,
                "end": 518
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "32925714",
            "location": {
                "start": 788,
                "end": 792
            }
        },
        {
            "tag": "group1",
            "row_uid": "32862601",
            "location": {
                "start": 870,
                "end": 879
            }
        },
        {
            "tag": "group2",
            "row_uid": "32862601",
            "location": {
                "start": 894,
                "end": 913
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32862601",
            "location": {
                "start": 463,
                "end": 474
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "32862601",
            "location": {
                "start": 1913,
                "end": 1917
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "32862601",
            "location": {
                "start": 1913,
                "end": 1917
            }
        },
        {
            "tag": "group1",
            "row_uid": "32850872",
            "location": {
                "start": 841,
                "end": 850
            }
        },
        {
            "tag": "group2",
            "row_uid": "32850872",
            "location": {
                "start": 900,
                "end": 936
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32850872",
            "location": {
                "start": 626,
                "end": 632
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "32850872",
            "location": {
                "start": 763,
                "end": 767
            }
        },
        {
            "tag": "group1",
            "row_uid": "32824907",
            "location": {
                "start": 196,
                "end": 205
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32824907",
            "location": {
                "start": 960,
                "end": 963
            }
        },
        {
            "tag": "group1",
            "row_uid": "32798360",
            "location": {
                "start": 642,
                "end": 672
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32798360",
            "location": {
                "start": 563,
                "end": 575
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "32798360",
            "location": {
                "start": 1342,
                "end": 1347
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32758341",
            "location": {
                "start": 792,
                "end": 796
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "32758341",
            "location": {
                "start": 1079,
                "end": 1083
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32748714",
            "location": {
                "start": 861,
                "end": 865
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "32748714",
            "location": {
                "start": 1579,
                "end": 1583
            }
        },
        {
            "tag": "group1",
            "row_uid": "32717852",
            "location": {
                "start": 647,
                "end": 656
            }
        },
        {
            "tag": "group2",
            "row_uid": "32717852",
            "location": {
                "start": 671,
                "end": 684
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32717852",
            "location": {
                "start": 511,
                "end": 524
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "32717852",
            "location": {
                "start": 534,
                "end": 538
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "32717852",
            "location": {
                "start": 551,
                "end": 554
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "32717852",
            "location": {
                "start": 1126,
                "end": 1130
            }
        },
        {
            "tag": "group1",
            "row_uid": "32601620",
            "location": {
                "start": 755,
                "end": 813
            }
        },
        {
            "tag": "group2",
            "row_uid": "32601620",
            "location": {
                "start": 818,
                "end": 836
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32601620",
            "location": {
                "start": 1354,
                "end": 1356
            }
        },
        {
            "tag": "group1",
            "row_uid": "32592092",
            "location": {
                "start": 488,
                "end": 497
            }
        },
        {
            "tag": "group2",
            "row_uid": "32592092",
            "location": {
                "start": 581,
                "end": 615
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32592092",
            "location": {
                "start": 757,
                "end": 759
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "32592092",
            "location": {
                "start": 1134,
                "end": 1137
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32580502",
            "location": {
                "start": 603,
                "end": 606
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "32580502",
            "location": {
                "start": 1381,
                "end": 1386
            }
        },
        {
            "tag": "group1",
            "row_uid": "32523648",
            "location": {
                "start": 746,
                "end": 761
            }
        },
        {
            "tag": "group2",
            "row_uid": "32523648",
            "location": {
                "start": 778,
                "end": 787
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32523648",
            "location": {
                "start": 967,
                "end": 969
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "32523648",
            "location": {
                "start": 980,
                "end": 982
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "32523648",
            "location": {
                "start": 980,
                "end": 982
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "32523648",
            "location": {
                "start": 1044,
                "end": 1047
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "32523648",
            "location": {
                "start": 1044,
                "end": 1047
            }
        },
        {
            "tag": "group1",
            "row_uid": "32478629",
            "location": {
                "start": 282,
                "end": 291
            }
        },
        {
            "tag": "group2",
            "row_uid": "32478629",
            "location": {
                "start": 342,
                "end": 349
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32478629",
            "location": {
                "start": 752,
                "end": 757
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32406122",
            "location": {
                "start": 856,
                "end": 864
            }
        },
        {
            "tag": "group1",
            "row_uid": "32384406",
            "location": {
                "start": 365,
                "end": 374
            }
        },
        {
            "tag": "group2",
            "row_uid": "32384406",
            "location": {
                "start": 885,
                "end": 905
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32384406",
            "location": {
                "start": 663,
                "end": 666
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "32384406",
            "location": {
                "start": 877,
                "end": 879
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "32384406",
            "location": {
                "start": 911,
                "end": 913
            }
        },
        {
            "tag": "response_p",
            "row_uid": "32384406",
            "location": {
                "start": 1177,
                "end": 1182
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "32369516",
            "location": {
                "start": 821,
                "end": 823
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "32369516",
            "location": {
                "start": 838,
                "end": 841
            }
        },
        {
            "tag": "group1",
            "row_uid": "32369446",
            "location": {
                "start": 649,
                "end": 731
            }
        },
        {
            "tag": "group2",
            "row_uid": "32369446",
            "location": {
                "start": 739,
                "end": 847
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32369446",
            "location": {
                "start": 546,
                "end": 567
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "32369446",
            "location": {
                "start": 1343,
                "end": 1354
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "32369446",
            "location": {
                "start": 1215,
                "end": 1226
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32367394",
            "location": {
                "start": 462,
                "end": 465
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "32367394",
            "location": {
                "start": 1193,
                "end": 1196
            }
        },
        {
            "tag": "g1_survival_rate",
            "row_uid": "32367394",
            "location": {
                "start": 1342,
                "end": 1347
            }
        },
        {
            "tag": "g2_survival_rate",
            "row_uid": "32367394",
            "location": {
                "start": 1352,
                "end": 1358
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "32367394",
            "location": {
                "start": 1699,
                "end": 1704
            }
        },
        {
            "tag": "survival_p",
            "row_uid": "32367394",
            "location": {
                "start": 1377,
                "end": 1382
            }
        },
        {
            "tag": "group2",
            "row_uid": "32304100",
            "location": {
                "start": 459,
                "end": 504
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32304100",
            "location": {
                "start": 862,
                "end": 866
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "32304100",
            "location": {
                "start": 1327,
                "end": 1331
            }
        },
        {
            "tag": "group2",
            "row_uid": "32296640",
            "location": {
                "start": 829,
                "end": 855
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32296640",
            "location": {
                "start": 973,
                "end": 979
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "32296640",
            "location": {
                "start": 1127,
                "end": 1131
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "32296640",
            "location": {
                "start": 1559,
                "end": 1563
            }
        },
        {
            "tag": "group1",
            "row_uid": "32274150",
            "location": {
                "start": 568,
                "end": 577
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32274150",
            "location": {
                "start": 739,
                "end": 744
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "32274150",
            "location": {
                "start": 1576,
                "end": 1579
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "32274150",
            "location": {
                "start": 1612,
                "end": 1615
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "32274150",
            "location": {
                "start": 1291,
                "end": 1296
            }
        },
        {
            "tag": "group1",
            "row_uid": "32271517",
            "location": {
                "start": 844,
                "end": 851
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32271517",
            "location": {
                "start": 697,
                "end": 701
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32258978",
            "location": {
                "start": 773,
                "end": 775
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "32258978",
            "location": {
                "start": 1463,
                "end": 1467
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32222327",
            "location": {
                "start": 874,
                "end": 877
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "32222327",
            "location": {
                "start": 1060,
                "end": 1063
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "32222327",
            "location": {
                "start": 1227,
                "end": 1230
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "32222327",
            "location": {
                "start": 1071,
                "end": 1077
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "32222327",
            "location": {
                "start": 1327,
                "end": 1336
            }
        },
        {
            "tag": "g2_median_OS",
            "row_uid": "32222327",
            "location": {
                "start": 1340,
                "end": 1349
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "32222327",
            "location": {
                "start": 1385,
                "end": 1389
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "32222327",
            "location": {
                "start": 1395,
                "end": 1400
            }
        },
        {
            "tag": "group1",
            "row_uid": "32200486",
            "location": {
                "start": 365,
                "end": 392
            }
        },
        {
            "tag": "group2",
            "row_uid": "32200486",
            "location": {
                "start": 398,
                "end": 416
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32200486",
            "location": {
                "start": 770,
                "end": 781
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "32200486",
            "location": {
                "start": 1209,
                "end": 1214
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "32200486",
            "location": {
                "start": 1209,
                "end": 1214
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32196659",
            "location": {
                "start": 364,
                "end": 370
            }
        },
        {
            "tag": "group1",
            "row_uid": "32190995",
            "location": {
                "start": 789,
                "end": 806
            }
        },
        {
            "tag": "group2",
            "row_uid": "32190995",
            "location": {
                "start": 828,
                "end": 854
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32190995",
            "location": {
                "start": 729,
                "end": 741
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "32190995",
            "location": {
                "start": 778,
                "end": 781
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "32190995",
            "location": {
                "start": 817,
                "end": 820
            }
        },
        {
            "tag": "g1_survival_rate",
            "row_uid": "32190995",
            "location": {
                "start": 948,
                "end": 953
            }
        },
        {
            "tag": "g2_survival_rate",
            "row_uid": "32190995",
            "location": {
                "start": 1005,
                "end": 1011
            }
        },
        {
            "tag": "group1",
            "row_uid": "32162820",
            "location": {
                "start": 554,
                "end": 599
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32162820",
            "location": {
                "start": 619,
                "end": 622
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "32162820",
            "location": {
                "start": 966,
                "end": 972
            }
        },
        {
            "tag": "g2_response_rate",
            "row_uid": "32162820",
            "location": {
                "start": 966,
                "end": 972
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "32162820",
            "location": {
                "start": 1161,
                "end": 1164
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "32162820",
            "location": {
                "start": 1169,
                "end": 1179
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "32162820",
            "location": {
                "start": 1185,
                "end": 1189
            }
        },
        {
            "tag": "group1",
            "row_uid": "32157435",
            "location": {
                "start": 661,
                "end": 670
            }
        },
        {
            "tag": "group2",
            "row_uid": "32157435",
            "location": {
                "start": 678,
                "end": 685
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32157435",
            "location": {
                "start": 991,
                "end": 994
            }
        },
        {
            "tag": "group1",
            "row_uid": "32156062",
            "location": {
                "start": 376,
                "end": 391
            }
        },
        {
            "tag": "group2",
            "row_uid": "32156062",
            "location": {
                "start": 461,
                "end": 479
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32156062",
            "location": {
                "start": 326,
                "end": 332
            }
        },
        {
            "tag": "group1",
            "row_uid": "32146090",
            "location": {
                "start": 718,
                "end": 727
            }
        },
        {
            "tag": "group2",
            "row_uid": "32146090",
            "location": {
                "start": 747,
                "end": 766
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32146090",
            "location": {
                "start": 612,
                "end": 615
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "32146090",
            "location": {
                "start": 697,
                "end": 699
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "32146090",
            "location": {
                "start": 732,
                "end": 734
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "32146090",
            "location": {
                "start": 852,
                "end": 864
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "32146090",
            "location": {
                "start": 955,
                "end": 966
            }
        },
        {
            "tag": "group1",
            "row_uid": "32114477",
            "location": {
                "start": 637,
                "end": 685
            }
        },
        {
            "tag": "group2",
            "row_uid": "32114477",
            "location": {
                "start": 1030,
                "end": 1041
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32114477",
            "location": {
                "start": 1417,
                "end": 1420
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32100113",
            "location": {
                "start": 781,
                "end": 782
            }
        },
        {
            "tag": "group1",
            "row_uid": "32085863",
            "location": {
                "start": 342,
                "end": 394
            }
        },
        {
            "tag": "group2",
            "row_uid": "32085863",
            "location": {
                "start": 398,
                "end": 408
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32085863",
            "location": {
                "start": 1001,
                "end": 1011
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "32085863",
            "location": {
                "start": 1041,
                "end": 1043
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "32085863",
            "location": {
                "start": 1082,
                "end": 1084
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "32085863",
            "location": {
                "start": 1032,
                "end": 1035
            }
        },
        {
            "tag": "g2_response_rate",
            "row_uid": "32085863",
            "location": {
                "start": 1073,
                "end": 1076
            }
        },
        {
            "tag": "response_p",
            "row_uid": "32085863",
            "location": {
                "start": 1396,
                "end": 1400
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32067604",
            "location": {
                "start": 877,
                "end": 882
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "32067604",
            "location": {
                "start": 1045,
                "end": 1049
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32060719",
            "location": {
                "start": 868,
                "end": 870
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "32060719",
            "location": {
                "start": 1339,
                "end": 1344
            }
        },
        {
            "tag": "group1",
            "row_uid": "32035002",
            "location": {
                "start": 428,
                "end": 448
            }
        },
        {
            "tag": "group2",
            "row_uid": "32035002",
            "location": {
                "start": 1055,
                "end": 1064
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32035002",
            "location": {
                "start": 983,
                "end": 989
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "32035002",
            "location": {
                "start": 1203,
                "end": 1207
            }
        },
        {
            "tag": "group1",
            "row_uid": "32021570",
            "location": {
                "start": 1186,
                "end": 1193
            }
        },
        {
            "tag": "group2",
            "row_uid": "32021570",
            "location": {
                "start": 655,
                "end": 691
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "32021570",
            "location": {
                "start": 770,
                "end": 782
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "32021570",
            "location": {
                "start": 1155,
                "end": 1158
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "32021570",
            "location": {
                "start": 1232,
                "end": 1234
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "32021570",
            "location": {
                "start": 1195,
                "end": 1415
            }
        },
        {
            "tag": "group1",
            "row_uid": "31990759",
            "location": {
                "start": 693,
                "end": 702
            }
        },
        {
            "tag": "group2",
            "row_uid": "31990759",
            "location": {
                "start": 710,
                "end": 717
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31990759",
            "location": {
                "start": 1131,
                "end": 1133
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "31990759",
            "location": {
                "start": 1175,
                "end": 1176
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "31990759",
            "location": {
                "start": 1591,
                "end": 1594
            }
        },
        {
            "tag": "group1",
            "row_uid": "31980358",
            "location": {
                "start": 284,
                "end": 876
            }
        },
        {
            "tag": "group2",
            "row_uid": "31980358",
            "location": {
                "start": 953,
                "end": 1064
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31980358",
            "location": {
                "start": 742,
                "end": 745
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31980358",
            "location": {
                "start": 849,
                "end": 852
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "31980358",
            "location": {
                "start": 1016,
                "end": 1018
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "31980358",
            "location": {
                "start": 1233,
                "end": 1240
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "31980358",
            "location": {
                "start": 1317,
                "end": 1324
            }
        },
        {
            "tag": "group1",
            "row_uid": "31913209",
            "location": {
                "start": 528,
                "end": 537
            }
        },
        {
            "tag": "group2",
            "row_uid": "31913209",
            "location": {
                "start": 607,
                "end": 653
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31913209",
            "location": {
                "start": 760,
                "end": 772
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31913209",
            "location": {
                "start": 793,
                "end": 795
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31896527",
            "location": {
                "start": 561,
                "end": 565
            }
        },
        {
            "tag": "group1",
            "row_uid": "31835708",
            "location": {
                "start": 919,
                "end": 954
            }
        },
        {
            "tag": "group2",
            "row_uid": "31835708",
            "location": {
                "start": 1068,
                "end": 1085
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31818523",
            "location": {
                "start": 1155,
                "end": 1162
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "31818523",
            "location": {
                "start": 1447,
                "end": 1451
            }
        },
        {
            "tag": "group1",
            "row_uid": "31787317",
            "location": {
                "start": 335,
                "end": 357
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31787317",
            "location": {
                "start": 807,
                "end": 810
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "31787317",
            "location": {
                "start": 844,
                "end": 850
            }
        },
        {
            "tag": "group1",
            "row_uid": "31764404",
            "location": {
                "start": 927,
                "end": 942
            }
        },
        {
            "tag": "group2",
            "row_uid": "31764404",
            "location": {
                "start": 960,
                "end": 978
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31764404",
            "location": {
                "start": 677,
                "end": 680
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31764404",
            "location": {
                "start": 948,
                "end": 950
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "31764404",
            "location": {
                "start": 984,
                "end": 987
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "31764404",
            "location": {
                "start": 1526,
                "end": 1531
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "31764404",
            "location": {
                "start": 1316,
                "end": 1321
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31752752",
            "location": {
                "start": 811,
                "end": 813
            }
        },
        {
            "tag": "group1",
            "row_uid": "31693512",
            "location": {
                "start": 457,
                "end": 466
            }
        },
        {
            "tag": "group2",
            "row_uid": "31693512",
            "location": {
                "start": 476,
                "end": 489
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31693512",
            "location": {
                "start": 338,
                "end": 351
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31693512",
            "location": {
                "start": 470,
                "end": 473
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "31693512",
            "location": {
                "start": 493,
                "end": 496
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31685560",
            "location": {
                "start": 556,
                "end": 561
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "31685560",
            "location": {
                "start": 1176,
                "end": 1180
            }
        },
        {
            "tag": "group1",
            "row_uid": "31645894",
            "location": {
                "start": 427,
                "end": 447
            }
        },
        {
            "tag": "group2",
            "row_uid": "31645894",
            "location": {
                "start": 451,
                "end": 468
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31645894",
            "location": {
                "start": 368,
                "end": 370
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "31645894",
            "location": {
                "start": 631,
                "end": 635
            }
        },
        {
            "tag": "g2_response_rate",
            "row_uid": "31645894",
            "location": {
                "start": 639,
                "end": 645
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "31645894",
            "location": {
                "start": 781,
                "end": 785
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "31645894",
            "location": {
                "start": 813,
                "end": 816
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "31645894",
            "location": {
                "start": 813,
                "end": 816
            }
        },
        {
            "tag": "response_p",
            "row_uid": "31645894",
            "location": {
                "start": 677,
                "end": 680
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "31645894",
            "location": {
                "start": 805,
                "end": 808
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "31645894",
            "location": {
                "start": 836,
                "end": 839
            }
        },
        {
            "tag": "group1",
            "row_uid": "31628524",
            "location": {
                "start": 438,
                "end": 457
            }
        },
        {
            "tag": "group2",
            "row_uid": "31628524",
            "location": {
                "start": 491,
                "end": 550
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31628524",
            "location": {
                "start": 663,
                "end": 674
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31628524",
            "location": {
                "start": 663,
                "end": 665
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "31628524",
            "location": {
                "start": 712,
                "end": 714
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "31628524",
            "location": {
                "start": 929,
                "end": 933
            }
        },
        {
            "tag": "group1",
            "row_uid": "31628143",
            "location": {
                "start": 546,
                "end": 582
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31628143",
            "location": {
                "start": 1114,
                "end": 1132
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "31628143",
            "location": {
                "start": 1300,
                "end": 1350
            }
        },
        {
            "tag": "g2_response_rate",
            "row_uid": "31628143",
            "location": {
                "start": 1366,
                "end": 1369
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "31628143",
            "location": {
                "start": 1551,
                "end": 1562
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "31628143",
            "location": {
                "start": 1496,
                "end": 1499
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "31628143",
            "location": {
                "start": 1496,
                "end": 1499
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "31628143",
            "location": {
                "start": 1671,
                "end": 1676
            }
        },
        {
            "tag": "group1",
            "row_uid": "31626727",
            "location": {
                "start": 779,
                "end": 796
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31626727",
            "location": {
                "start": 1195,
                "end": 1210
            }
        },
        {
            "tag": "g1_survival_rate",
            "row_uid": "31626727",
            "location": {
                "start": 1802,
                "end": 1806
            }
        },
        {
            "tag": "g2_survival_rate",
            "row_uid": "31626727",
            "location": {
                "start": 1802,
                "end": 1806
            }
        },
        {
            "tag": "group1",
            "row_uid": "31583436",
            "location": {
                "start": 749,
                "end": 769
            }
        },
        {
            "tag": "group2",
            "row_uid": "31583436",
            "location": {
                "start": 775,
                "end": 799
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31583436",
            "location": {
                "start": 1268,
                "end": 1280
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31583436",
            "location": {
                "start": 1296,
                "end": 1298
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "31583436",
            "location": {
                "start": 1318,
                "end": 1320
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "31583436",
            "location": {
                "start": 1405,
                "end": 1409
            }
        },
        {
            "tag": "g2_response_rate",
            "row_uid": "31583436",
            "location": {
                "start": 1414,
                "end": 1418
            }
        },
        {
            "tag": "group1",
            "row_uid": "31570268",
            "location": {
                "start": 495,
                "end": 518
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31570268",
            "location": {
                "start": 1092,
                "end": 1106
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "31570268",
            "location": {
                "start": 1602,
                "end": 1609
            }
        },
        {
            "tag": "group1",
            "row_uid": "31518336",
            "location": {
                "start": 576,
                "end": 591
            }
        },
        {
            "tag": "group2",
            "row_uid": "31518336",
            "location": {
                "start": 692,
                "end": 1176
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31518336",
            "location": {
                "start": 865,
                "end": 878
            }
        },
        {
            "tag": "group1",
            "row_uid": "31496802",
            "location": {
                "start": 357,
                "end": 804
            }
        },
        {
            "tag": "group2",
            "row_uid": "31496802",
            "location": {
                "start": 414,
                "end": 423
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31496802",
            "location": {
                "start": 906,
                "end": 919
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31491730",
            "location": {
                "start": 654,
                "end": 660
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "31491730",
            "location": {
                "start": 1627,
                "end": 1631
            }
        },
        {
            "tag": "group1",
            "row_uid": "31486833",
            "location": {
                "start": 1833,
                "end": 1842
            }
        },
        {
            "tag": "group2",
            "row_uid": "31486833",
            "location": {
                "start": 1356,
                "end": 1879
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31486833",
            "location": {
                "start": 1737,
                "end": 1749
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31486833",
            "location": {
                "start": 1848,
                "end": 1850
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "31486833",
            "location": {
                "start": 1885,
                "end": 1887
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "31486833",
            "location": {
                "start": 2207,
                "end": 2211
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "31486833",
            "location": {
                "start": 1969,
                "end": 1973
            }
        },
        {
            "tag": "g2_median_OS",
            "row_uid": "31486833",
            "location": {
                "start": 2234,
                "end": 2238
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "31486833",
            "location": {
                "start": 2035,
                "end": 2038
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "31486833",
            "location": {
                "start": 2080,
                "end": 2084
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "31486833",
            "location": {
                "start": 2080,
                "end": 2084
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "31486833",
            "location": {
                "start": 2270,
                "end": 2273
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "31486833",
            "location": {
                "start": 2109,
                "end": 2112
            }
        },
        {
            "tag": "group1",
            "row_uid": "31472446",
            "location": {
                "start": 613,
                "end": 635
            }
        },
        {
            "tag": "group2",
            "row_uid": "31472446",
            "location": {
                "start": 422,
                "end": 429
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31472446",
            "location": {
                "start": 945,
                "end": 947
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31472446",
            "location": {
                "start": 974,
                "end": 976
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "31472446",
            "location": {
                "start": 988,
                "end": 990
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "31472446",
            "location": {
                "start": 1306,
                "end": 1311
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "31472446",
            "location": {
                "start": 1535,
                "end": 1539
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "31472446",
            "location": {
                "start": 1454,
                "end": 1457
            }
        },
        {
            "tag": "g2_median_OS",
            "row_uid": "31472446",
            "location": {
                "start": 1555,
                "end": 1566
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "31472446",
            "location": {
                "start": 1461,
                "end": 1471
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "31472446",
            "location": {
                "start": 1581,
                "end": 1585
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "31472446",
            "location": {
                "start": 897,
                "end": 901
            }
        },
        {
            "tag": "group1",
            "row_uid": "31470817",
            "location": {
                "start": 757,
                "end": 766
            }
        },
        {
            "tag": "group2",
            "row_uid": "31470817",
            "location": {
                "start": 788,
                "end": 805
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31470817",
            "location": {
                "start": 504,
                "end": 507
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31470817",
            "location": {
                "start": 740,
                "end": 742
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "31470817",
            "location": {
                "start": 776,
                "end": 778
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "31470817",
            "location": {
                "start": 1212,
                "end": 1217
            }
        },
        {
            "tag": "group1",
            "row_uid": "31413010",
            "location": {
                "start": 705,
                "end": 736
            }
        },
        {
            "tag": "group2",
            "row_uid": "31413010",
            "location": {
                "start": 740,
                "end": 747
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31413010",
            "location": {
                "start": 579,
                "end": 582
            }
        },
        {
            "tag": "g2_response_rate",
            "row_uid": "31413010",
            "location": {
                "start": 1424,
                "end": 1431
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "31413010",
            "location": {
                "start": 1294,
                "end": 1305
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "31413010",
            "location": {
                "start": 1250,
                "end": 1261
            }
        },
        {
            "tag": "g2_median_OS",
            "row_uid": "31413010",
            "location": {
                "start": 1310,
                "end": 1321
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "31413010",
            "location": {
                "start": 1250,
                "end": 1261
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "31413010",
            "location": {
                "start": 1229,
                "end": 1235
            }
        },
        {
            "tag": "group1",
            "row_uid": "31409137",
            "location": {
                "start": 632,
                "end": 641
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31409137",
            "location": {
                "start": 628,
                "end": 631
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "31409137",
            "location": {
                "start": 1067,
                "end": 1078
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "31409137",
            "location": {
                "start": 959,
                "end": 969
            }
        },
        {
            "tag": "g2_median_OS",
            "row_uid": "31409137",
            "location": {
                "start": 1102,
                "end": 1113
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "31409137",
            "location": {
                "start": 992,
                "end": 1002
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "31409137",
            "location": {
                "start": 1145,
                "end": 1150
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "31409137",
            "location": {
                "start": 1032,
                "end": 1036
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31398539",
            "location": {
                "start": 880,
                "end": 883
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31398539",
            "location": {
                "start": 978,
                "end": 980
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "31398539",
            "location": {
                "start": 1144,
                "end": 1148
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "31398539",
            "location": {
                "start": 1236,
                "end": 1240
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "31398539",
            "location": {
                "start": 1154,
                "end": 1158
            }
        },
        {
            "tag": "group1",
            "row_uid": "31345259",
            "location": {
                "start": 1179,
                "end": 1188
            }
        },
        {
            "tag": "group2",
            "row_uid": "31345259",
            "location": {
                "start": 1203,
                "end": 1222
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31345259",
            "location": {
                "start": 1119,
                "end": 1122
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31345259",
            "location": {
                "start": 1175,
                "end": 1178
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "31345259",
            "location": {
                "start": 1199,
                "end": 1202
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31333095",
            "location": {
                "start": 821,
                "end": 826
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "31333095",
            "location": {
                "start": 930,
                "end": 934
            }
        },
        {
            "tag": "group1",
            "row_uid": "31324361",
            "location": {
                "start": 1019,
                "end": 1029
            }
        },
        {
            "tag": "group2",
            "row_uid": "31324361",
            "location": {
                "start": 1051,
                "end": 1063
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31300371",
            "location": {
                "start": 947,
                "end": 950
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31293011",
            "location": {
                "start": 1256,
                "end": 1260
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31192140",
            "location": {
                "start": 731,
                "end": 736
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "31192140",
            "location": {
                "start": 961,
                "end": 965
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "31192140",
            "location": {
                "start": 1348,
                "end": 1352
            }
        },
        {
            "tag": "g1_survival_rate",
            "row_uid": "31192140",
            "location": {
                "start": 1283,
                "end": 1289
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "31192140",
            "location": {
                "start": 1390,
                "end": 1395
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31139216",
            "location": {
                "start": 451,
                "end": 454
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "31135975",
            "location": {
                "start": 400,
                "end": 403
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "31135975",
            "location": {
                "start": 1378,
                "end": 1382
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "31124302",
            "location": {
                "start": 915,
                "end": 919
            }
        },
        {
            "tag": "Other_HR",
            "row_uid": "31124302",
            "location": {
                "start": 915,
                "end": 919
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "31012989",
            "location": {
                "start": 1020,
                "end": 1024
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30999871",
            "location": {
                "start": 388,
                "end": 391
            }
        },
        {
            "tag": "g1_survival_rate",
            "row_uid": "30999871",
            "location": {
                "start": 1503,
                "end": 1509
            }
        },
        {
            "tag": "g2_survival_rate",
            "row_uid": "30999871",
            "location": {
                "start": 1526,
                "end": 1532
            }
        },
        {
            "tag": "group1",
            "row_uid": "30984619",
            "location": {
                "start": 792,
                "end": 871
            }
        },
        {
            "tag": "group2",
            "row_uid": "30984619",
            "location": {
                "start": 894,
                "end": 911
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30984619",
            "location": {
                "start": 630,
                "end": 632
            }
        },
        {
            "tag": "response_p",
            "row_uid": "30984619",
            "location": {
                "start": 1646,
                "end": 1651
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30980539",
            "location": {
                "start": 418,
                "end": 423
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "30980539",
            "location": {
                "start": 1415,
                "end": 1419
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "30980539",
            "location": {
                "start": 1147,
                "end": 1151
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30827861",
            "location": {
                "start": 821,
                "end": 825
            }
        },
        {
            "tag": "group1",
            "row_uid": "30803422",
            "location": {
                "start": 760,
                "end": 769
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30803422",
            "location": {
                "start": 617,
                "end": 620
            }
        },
        {
            "tag": "group1",
            "row_uid": "30787638",
            "location": {
                "start": 245,
                "end": 252
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30787638",
            "location": {
                "start": 812,
                "end": 824
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30778081",
            "location": {
                "start": 767,
                "end": 776
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "30778081",
            "location": {
                "start": 897,
                "end": 901
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30686756",
            "location": {
                "start": 371,
                "end": 382
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30664988",
            "location": {
                "start": 491,
                "end": 495
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "30664988",
            "location": {
                "start": 920,
                "end": 924
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "30664988",
            "location": {
                "start": 1056,
                "end": 1060
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30663998",
            "location": {
                "start": 1038,
                "end": 1042
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "30663998",
            "location": {
                "start": 1391,
                "end": 1395
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "30663998",
            "location": {
                "start": 1278,
                "end": 1282
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30663219",
            "location": {
                "start": 546,
                "end": 585
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "30663219",
            "location": {
                "start": 964,
                "end": 967
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "30663219",
            "location": {
                "start": 1391,
                "end": 1393
            }
        },
        {
            "tag": "g2_median_OS",
            "row_uid": "30663219",
            "location": {
                "start": 1397,
                "end": 1406
            }
        },
        {
            "tag": "OS_p",
            "row_uid": "30663219",
            "location": {
                "start": 1412,
                "end": 1417
            }
        },
        {
            "tag": "group2",
            "row_uid": "30655986",
            "location": {
                "start": 1044,
                "end": 1074
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30655986",
            "location": {
                "start": 1076,
                "end": 1095
            }
        },
        {
            "tag": "g1_response_rate",
            "row_uid": "30655986",
            "location": {
                "start": 1151,
                "end": 1156
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "30655986",
            "location": {
                "start": 1457,
                "end": 1461
            }
        },
        {
            "tag": "group1",
            "row_uid": "30617039",
            "location": {
                "start": 1494,
                "end": 1567
            }
        },
        {
            "tag": "group2",
            "row_uid": "30617039",
            "location": {
                "start": 1579,
                "end": 1597
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "30617039",
            "location": {
                "start": 1926,
                "end": 1936
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "30617039",
            "location": {
                "start": 1982,
                "end": 1991
            }
        },
        {
            "tag": "group1",
            "row_uid": "30610588",
            "location": {
                "start": 865,
                "end": 901
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30610588",
            "location": {
                "start": 1028,
                "end": 1047
            }
        },
        {
            "tag": "g1_median_OS",
            "row_uid": "30610588",
            "location": {
                "start": 1381,
                "end": 1392
            }
        },
        {
            "tag": "g1_median_PFS",
            "row_uid": "30610588",
            "location": {
                "start": 1318,
                "end": 1328
            }
        },
        {
            "tag": "g2_median_OS",
            "row_uid": "30610588",
            "location": {
                "start": 1381,
                "end": 1392
            }
        },
        {
            "tag": "g2_median_PFS",
            "row_uid": "30610588",
            "location": {
                "start": 1381,
                "end": 1392
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "30610588",
            "location": {
                "start": 1689,
                "end": 1694
            }
        },
        {
            "tag": "group2",
            "row_uid": "30606884",
            "location": {
                "start": 680,
                "end": 730
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30606884",
            "location": {
                "start": 989,
                "end": 994
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "30606884",
            "location": {
                "start": 1217,
                "end": 1219
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "30606884",
            "location": {
                "start": 1283,
                "end": 1285
            }
        },
        {
            "tag": "OS_HR",
            "row_uid": "30606884",
            "location": {
                "start": 1482,
                "end": 1486
            }
        },
        {
            "tag": "PFS_HR",
            "row_uid": "30606884",
            "location": {
                "start": 1558,
                "end": 1562
            }
        },
        {
            "tag": "PFS_p",
            "row_uid": "30606884",
            "location": {
                "start": 1587,
                "end": 1592
            }
        },
        {
            "tag": "group1",
            "row_uid": "30588519",
            "location": {
                "start": 538,
                "end": 555
            }
        },
        {
            "tag": "group2",
            "row_uid": "30588519",
            "location": {
                "start": 618,
                "end": 627
            }
        },
        {
            "tag": "total_sample_size",
            "row_uid": "30588519",
            "location": {
                "start": 559,
                "end": 562
            }
        },
        {
            "tag": "g1_n",
            "row_uid": "30588519",
            "location": {
                "start": 559,
                "end": 562
            }
        },
        {
            "tag": "g2_n",
            "row_uid": "30588519",
            "location": {
                "start": 579,
                "end": 581
            }
        }
    ]
}
sample_request['data']

[{'tag': 'total_sample_size',
  'row_uid': '33278301',
  'location': {'start': 762, 'end': 765}},
 {'tag': 'g1_n',
  'row_uid': '33278301',
  'location': {'start': 1183, 'end': 1185}},
 {'tag': 'g2_n',
  'row_uid': '33278301',
  'location': {'start': 1208, 'end': 1210}},
 {'tag': 'total_sample_size',
  'row_uid': '33225411',
  'location': {'start': 1107, 'end': 1119}},
 {'tag': 'group1',
  'row_uid': '32642699',
  'location': {'start': 577, 'end': 595}},
 {'tag': 'group2',
  'row_uid': '32642699',
  'location': {'start': 577, 'end': 595}},
 {'tag': 'total_sample_size',
  'row_uid': '32642699',
  'location': {'start': 1162, 'end': 1181}},
 {'tag': 'g1_response_rate',
  'row_uid': '32642699',
  'location': {'start': 1310, 'end': 1315}},
 {'tag': 'g2_response_rate',
  'row_uid': '32642699',
  'location': {'start': 1386, 'end': 1391}},
 {'tag': 'g1_median_OS',
  'row_uid': '32642699',
  'location': {'start': 1457, 'end': 1460}},
 {'tag': 'g1_median_PFS',
  'row_uid': '32642699',
  'locatio

In [175]:
len(sample_request['data'])

565

In [176]:
annotations2 = []

In [177]:
for _sample in sample_request['data']:
    print(_sample)
    annotations2.append({
            "uuid": str(uuid.uuid4()), 
            "schemaId": get_entity_schema_id(ontology, _sample['tag'])['featureSchemaId'],
            "dataRow": { 
                "id": PMIDS[_sample['row_uid']]
            }, 
            "location": {
                "start": _sample['location']['start'],
                "end": _sample['location']['end'],
            }
    })
    print({
            "uuid": str(uuid.uuid4()), 
            "schemaId": get_entity_schema_id(ontology, _sample['tag'])['featureSchemaId'],
            "dataRow": { 
                "id": PMIDS[_sample['row_uid']]
            }, 
            "location": {
                "start": _sample['location']['start'],
                "end": _sample['location']['end'],
            }
    })
annotations2

{'tag': 'total_sample_size', 'row_uid': '33278301', 'location': {'start': 762, 'end': 765}}
{'uuid': '5ffec985-3e86-4e65-a160-a54a120b1bb9', 'schemaId': 'ckjrsj79q0d550y83cx113wp1', 'dataRow': {'id': 'ckl25lgi2074q0y8khtce97mi'}, 'location': {'start': 762, 'end': 765}}
{'tag': 'g1_n', 'row_uid': '33278301', 'location': {'start': 1183, 'end': 1185}}
{'uuid': 'd046ddb5-fcbf-4447-8a72-222e1a8c55d6', 'schemaId': 'ckjrsj79r0d560y83fv2875ab', 'dataRow': {'id': 'ckl25lgi2074q0y8khtce97mi'}, 'location': {'start': 1183, 'end': 1185}}
{'tag': 'g2_n', 'row_uid': '33278301', 'location': {'start': 1208, 'end': 1210}}
{'uuid': 'fdfda930-4c6b-4ad1-a594-50eca2388613', 'schemaId': 'ckjrsj79r0d570y83aw4xg9t7', 'dataRow': {'id': 'ckl25lgi2074q0y8khtce97mi'}, 'location': {'start': 1208, 'end': 1210}}
{'tag': 'total_sample_size', 'row_uid': '33225411', 'location': {'start': 1107, 'end': 1119}}
{'uuid': 'c9dc008c-9a06-4e64-a3a1-b6ea15921e82', 'schemaId': 'ckjrsj79q0d550y83cx113wp1', 'dataRow': {'id': 'ckl25

[{'uuid': '12381658-acbf-4688-a965-999f3dc277a4',
  'schemaId': 'ckjrsj79q0d550y83cx113wp1',
  'dataRow': {'id': 'ckl25lgi2074q0y8khtce97mi'},
  'location': {'start': 762, 'end': 765}},
 {'uuid': '1de4593a-1575-4d70-bc88-e0328d834390',
  'schemaId': 'ckjrsj79r0d560y83fv2875ab',
  'dataRow': {'id': 'ckl25lgi2074q0y8khtce97mi'},
  'location': {'start': 1183, 'end': 1185}},
 {'uuid': 'c1457571-5493-419c-903d-971b167c4f16',
  'schemaId': 'ckjrsj79r0d570y83aw4xg9t7',
  'dataRow': {'id': 'ckl25lgi2074q0y8khtce97mi'},
  'location': {'start': 1208, 'end': 1210}},
 {'uuid': '78750f44-8cb7-447d-a952-bd4a5852c380',
  'schemaId': 'ckjrsj79q0d550y83cx113wp1',
  'dataRow': {'id': 'ckl25lijk075g0y8k6t7zhr3p'},
  'location': {'start': 1107, 'end': 1119}},
 {'uuid': 'ff8ed361-19e5-49a3-b388-8a82adeb092a',
  'schemaId': 'ckjrshew80b7i0y7g9zzydsj4',
  'dataRow': {'id': 'ckl25lj8708av0y6x33te5hkn'},
  'location': {'start': 577, 'end': 595}},
 {'uuid': '356cfbe4-3cee-4590-9487-178b1c778e70',
  'schemaId': 

In [199]:
len(annotations2)

565

In [200]:
len(annotations2)

565

### Upload Annotation

In [201]:
upload_job = project.upload_annotations(
    name='upload_test_annotations_7', ## Needs to be unique
    annotations=annotations2)

upload_job.wait_until_done()
upload_job.state

<BulkImportRequestState.FINISHED: 'FINISHED'>

### Check Upload Status

In [57]:
import ndjson
import requests

print(f'Here is the status file: {upload_job.status_file_url}')
print(f'Here is the error file: {upload_job.error_file_url}')

res = requests.get(upload_job.status_file_url)
statuses = ndjson.loads(res.text)

statuses

Here is the status file: https://storage.googleapis.com/labelbox-predictions-import-prod/status/334c74d6-fffe-4109-bf68-ca788fcd426b.ndjson?GoogleAccessId=api-prod%40labelbox-193903.iam.gserviceaccount.com&Expires=1613216629&Signature=L9GbXJb1wd30ftbN8qhNgR8kCAYsEbR57bXnUOqHNWgTgAcGKL9SZrzM7D%2F19f5XHfO%2FIflxWRsKTrpnuUN77q6KGh2EZJtFPAqNNC8Zw4vg5V6q2D%2Fmf2IhiF0LhHBSzhfTKqovDinFkhLEQTEtzSsW1ACY07L1QrzqwvsYQE%2BwOwkUGhwW4w2PC6MqA%2Ft%2Bt48qQ5SaijcuV%2BY2BS7Bs72Yh0Lf6OMbsTqt9VGYFoneA5n9SMABqw9HEZeTex1lKhsef3LjBQLuJgorFA1T40CZ%2BZubb1QocDvOXqzhJatUBUfDAZVefxiALubBrOQTSKNcepnNGpQhdRRZ28uMZQ%3D%3D
Here is the error file: https://storage.googleapis.com/labelbox-predictions-import-prod/errors/334c74d6-fffe-4109-bf68-ca788fcd426b.ndjson?GoogleAccessId=api-prod%40labelbox-193903.iam.gserviceaccount.com&Expires=1613216629&Signature=dvzUcox6aaFp2KUd1APoljslCpiXfLIr%2BSLS0xAYTq2ZnBmUbBTIl20A1a98UmAAvp0Le9eY2n2%2BXloGyeNYTbRe3rzSHUOqQc%2Bd6swjtv6pGcWYpFU4myGgixcYATJr2Whu52czTUHr9Y0UDGy8cuhFU5XgU7f6

[{'uuid': '111b75d8-1ebd-42e4-ad2c-de16032454e2',
  'dataRow': {'id': 'ckl25lgi2074q0y8khtce97mi'},
  'status': 'SUCCESS'},
 {'uuid': '3aba9db9-3ad6-4201-9400-a7db1e98cb31',
  'dataRow': {'id': 'ckl25lgi2074q0y8khtce97mi'},
  'status': 'SUCCESS'},
 {'uuid': '7e07147f-d284-47de-ab4f-f80f7544931d',
  'dataRow': {'id': 'ckl25lgi2074q0y8khtce97mi'},
  'status': 'SUCCESS'},
 {'uuid': 'b4ca217b-9145-4f85-a144-b6e019b7747f',
  'dataRow': {'id': 'ckl25lijk075g0y8k6t7zhr3p'},
  'status': 'SUCCESS'},
 {'uuid': 'cdd8098c-495a-414c-8080-0da39742738a',
  'dataRow': {'id': 'ckl25lj8708av0y6x33te5hkn'},
  'status': 'SUCCESS'},
 {'uuid': '2b3f382d-f0cc-4634-896b-1b633f81171e',
  'dataRow': {'id': 'ckl25lj8708av0y6x33te5hkn'},
  'status': 'SUCCESS'},
 {'uuid': '810c316a-9194-4c5f-a7a1-3e38ff192a6a',
  'dataRow': {'id': 'ckl25lj8708av0y6x33te5hkn'},
  'status': 'SUCCESS'},
 {'uuid': 'bf00035f-2cfb-46ce-8c5b-a083c393974b',
  'dataRow': {'id': 'ckl25lj8708av0y6x33te5hkn'},
  'status': 'SUCCESS'},
 {'uuid'

### Get All Uploaded Annotation

In [184]:
all_import_requests = get_current_import_requests(project.uid)
all_import_requests

[{'id': 'ef6686fc-7cdc-4806-aaec-51159ba60471',
  'name': 'upload_test_annotations_6',
  'state': 'FINISHED'},
 {'id': '9745a2ea-de77-45d3-80d0-ce931cd78052',
  'name': 'upload_test_annotations_5',
  'state': 'FINISHED'},
 {'id': '334c74d6-fffe-4109-bf68-ca788fcd426b',
  'name': 'upload_test_annotations_4',
  'state': 'FINISHED'},
 {'id': 'b3b74b8f-4932-4e43-8cc7-e4c353f8a96b',
  'name': 'upload_test_annotations_3',
  'state': 'FINISHED'},
 {'id': '1d82e52f-0682-4c71-9b66-31f1a0fb6306',
  'name': 'test_rebootrx_data_rheyannmagcalas_1613127856',
  'state': 'FINISHED'},
 {'id': 'e1f494d9-bbb4-4368-a8e7-59dfc7910e86',
  'name': 'upload_test_annotations_1',
  'state': 'FINISHED'}]

### Delete Uploaded Annotation

In [ ]:
for task in all_import_requests:
    response = delete_import_request(task['id'])
    print(response)

In [190]:
response = delete_import_request('e1f494d9-bbb4-4368-a8e7-59dfc7910e86')
print(response)

{'deleteBulkImportRequest': {'id': 'e1f494d9-bbb4-4368-a8e7-59dfc7910e86', 'name': 'upload_test_annotations_1'}}


### Correlate Annotated Labels to Unlabelled Abstract

In [ ]:
data = next(iter(project.datasets()))
rows = list(data.data_rows())
for _rows in rows:
    for _label in _rows.labels():
        print(_rows.uid)
        print(_label)

In [ ]:
def turn_on_model_assisted_labeling(client: Client, project_id: str) -> None:
    """
    Turns model assisted labeling on for the given project

    Args:
        client (Client): The client that is connected via API key
        project_id (str): The id of the project
    Returns:
        None

    """
    client.execute("""
         mutation TurnPredictionsOn($proj_id: ID!){
             project(
                 where: {id: $proj_id}
             ){
                 showPredictionsToLabelers(show:true){
                     id
                     showingPredictionsToLabelers
                 }
             }
         }
     """, {"proj_id": project_id})

In [ ]:
turn_on_model_assisted_labeling(client, project.uid)